In [44]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

base_url = 'http://video.ittensive.com/data/018-python-advanced/beru.ru/'
r = requests.get(base_url)
html = BeautifulSoup(r.content)

# Мое решение
conn = sqlite3.connect('/home/eugeney/data.db3')
db = conn.cursor()

### Create table
# db.execute("""CREATE TABLE beru_goods
#     (id INTEGER PRIMARY KEY AUTOINCREMENT not null,
#     url text,
#     name text default '',
#     price integer default 0,
#     width real default 0.0,
#     depth real default 0.0,
#     height real default 0.0,
#     vol_tot integer default 0,
#     vol_cam integer default 0)""")
# conn.commit()

all_links = html.find_all('a')
links = []

# 
def get_data(url):
    r = requests.get(base_url + url)
    html = BeautifulSoup(r.content)

    name = html.find_all('meta', {'itemprop': 'name'})[2]['content']
    price = int(html.find('meta', {'itemprop': 'price'})['content'])
    width = 0
    depth = 0
    height = 0
    vol_tot = 0
    vol_cam = 0

    tags = html.find_all('span', {'class':'_112Tad-7AP'})
    for tag in tags:
        tag = tag.get_text()
        if tag.find('ШхВхГ') > -1:
            dims = tag.split(':')[1].split('х')
            width = float(dims[0])
            depth = float(dims[1])
            height = float(dims[2].split(' ')[0])
            size = tag
        if tag.find('общий объем') > -1:        
            vol_tot = find_number(tag)
        if tag.find('объем холодильной камеры') > -1:        
            vol_cam = find_number(tag)

    return [url, name, price, width, depth, height, vol_tot, vol_cam]

def find_number(s):
    return int('0' + ''.join(i for i in s if i.isdigit()))

### Insert data
data = []
for link in all_links:
    if str(link).find('Саратов') > -1:        
        url = link['href']
        print(url)
        data.append(get_data(url))
# print(data)

db.executemany("INSERT INTO beru_goods(url, name, price, width, depth, height, vol_tot, vol_cam) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", data)
conn.commit()  
db.close()      

## Select data
data = pd.read_sql_query('select * from beru_goods', conn)
print(data)

kholodilnik-saratov-452-ksh-120.html
kholodilnik-saratov-263-kshd-200-30.html
kholodilnik-saratov-209-kshd-275-65.html
kholodilnik-saratov-209-belyi-s-chernymi-nakladkami.html
kholodilnik-saratov-452-ksh-120-seryi.html
kholodilnik-saratov-550-ksh-120-bez-nto.html
kholodilnik-saratov-467-ksh-210.html
kholodilnik-saratov-264-kshd-150-30.html
kholodilnik-saratov-264-kshd-150-30-seryi.html
kholodilnik-saratov-263-seryi.html
[['kholodilnik-saratov-452-ksh-120.html', 'Холодильник Саратов 452 (КШ-120)', 10728, 48.0, 89.6, 60.0, 122, 107], ['kholodilnik-saratov-263-kshd-200-30.html', 'Холодильник Саратов 263 (КШД-200/30)', 14299, 48.0, 149.0, 60.0, 195, 165], ['kholodilnik-saratov-209-kshd-275-65.html', 'Холодильник Саратов 209 (КШД 275/65)', 13292, 60.0, 163.0, 60.6, 275, 210], ['kholodilnik-saratov-209-belyi-s-chernymi-nakladkami.html', 'Холодильник Саратов 209 белый с черными накладками', 18259, 60.0, 163.0, 61.0, 275, 210], ['kholodilnik-saratov-452-ksh-120-seryi.html', 'Холодильник Сарато